In [1]:
import os
import gc
import pickle
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)

2024-05-27 08:11:31.647102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 08:11:31.647192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 08:11:31.806084: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0
3.2.1


In [2]:
IMG_WIDTH = 180
IMG_HEIGHT = 150
EPOCHS = 50
BATCH_SIZE = 24
PATIENCE = 3
K_FOLD = 5
GC_COUNT = gc.collect()

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

Number of devices: 2


In [3]:
def data_read_preprocess(fold, train_data=True):
    if train_data:
        data_dir = f'/kaggle/input/imageoasis-{IMG_HEIGHT}x{IMG_WIDTH}-aug-5fold/fold_{fold}/train'
    else:
        data_dir = f'/kaggle/input/imageoasis-{IMG_HEIGHT}x{IMG_WIDTH}-aug-5fold/fold_{fold}/val'
        
    print(f'Reading data from: {data_dir}')
    data = tf.keras.preprocessing.image_dataset_from_directory(
            data_dir,
            labels="inferred", 
            label_mode="categorical", 
            color_mode="rgb", 
            batch_size=BATCH_SIZE,
            image_size=(IMG_HEIGHT, IMG_WIDTH))
    
    data = data.map(
        lambda x, y: (tf.keras.applications.xception.preprocess_input(x), y),
        num_parallel_calls=tf.data.AUTOTUNE)
    
    if train_data:
        data = data.shuffle(1000)
    
    data = data.prefetch(tf.data.AUTOTUNE)
    
    return data

In [4]:
def get_model(num_of_trainable):
    
    base_model = tf.keras.applications.xception.Xception(
        include_top=False,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
        weights='imagenet')
    
    base_model.trainable = True
    num_layers = len(base_model.layers)
    for layer in base_model.layers[:(num_layers - num_of_trainable)]:
        layer.trainable = False
    
    inputs = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D(name="GAPool2D")(x)
    x = tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, name="FC_1")(x)
    x = tf.keras.layers.Dropout(0.4, name="Drop_1")(x)
    outputs = tf.keras.layers.Dense(4, activation=tf.keras.activations.softmax, name="FC_classifier")(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy'])
    
    model.summary()
    
    return model

In [5]:
def get_callbacks(fold):
    callbacks = []
    
    es = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=PATIENCE,
        restore_best_weights=True)
    callbacks.append(es)
    
    c_log = tf.keras.callbacks.CSVLogger(
        filename=f'/kaggle/working/model_log_{IMG_HEIGHT}x{IMG_WIDTH}_{K_FOLD}fold/CSVLogger_fold{fold}.csv',
        append=True)
    callbacks.append(c_log)
    
    return callbacks

In [6]:
os.makedirs(f'/kaggle/working/model_log_{IMG_HEIGHT}x{IMG_WIDTH}_{K_FOLD}fold', exist_ok=True)

In [7]:
for x in range(1, (K_FOLD+1)):
    data_train = data_read_preprocess(x, train_data=True)
    data_val = data_read_preprocess(x, train_data=False)
    
    print(f"Training for fold-{x}:")
    with strategy.scope():
        model = get_model(6)
        
    history =  model.fit(data_train,
                         epochs=EPOCHS,
                         verbose=2,
                         callbacks=get_callbacks(x),
                         validation_data=data_val)
    
    model.save(f'/kaggle/working/model_log_{IMG_HEIGHT}x{IMG_WIDTH}_{K_FOLD}fold/model_fold-{x}.keras')
    with open(f'/kaggle/working/model_log_{IMG_HEIGHT}x{IMG_WIDTH}_{K_FOLD}fold/train_history_fold-{x}.pickle', 'wb') as handler:
        pickle.dump(history.history, handler)
        
    del data_train
    del data_val
    del model
    del history
    print("Garbage collector: collected %d objects." % (GC_COUNT))

Reading data from: /kaggle/input/imageoasis-150x180-aug-5fold/fold_1/train
Found 20000 files belonging to 4 classes.
Reading data from: /kaggle/input/imageoasis-150x180-aug-5fold/fold_1/val
Found 5000 files belonging to 4 classes.
Training for fold-1:
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 150, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 5, 6, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GAPool2D                        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC_1 (Dense)                    │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Drop_1 (Dropout)                │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC_classifier (Dense)           │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,912,620 (83.59 MB)

 Trainable params: 5,799,940 (22.13 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 150, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 5, 6, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GAPool2D                        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC_1 (Dense)                    │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Drop_1 (Dropout)                │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC_classifier (Dense)           │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,912,620 (83.59 MB)

 Trainable params: 5,799,940 (22.13 MB)

 Non-trainable params: 16,112,680 (61.46 MB)

Epoch 1/50
834/834 - 91s - 109ms/step - accuracy: 0.6803 - loss: 0.7622 - val_accuracy: 0.8208 - val_loss: 0.4624
Epoch 2/50
834/834 - 56s - 68ms/step - accuracy: 0.8201 - loss: 0.4584 - val_accuracy: 0.8668 - val_loss: 0.3385
Epoch 3/50
834/834 - 56s - 68ms/step - accuracy: 0.8723 - loss: 0.3426 - val_accuracy: 0.8900 - val_loss: 0.2998
Epoch 4/50
834/834 - 57s - 68ms/step - accuracy: 0.8995 - loss: 0.2747 - val_accuracy: 0.9128 - val_loss: 0.2280
Epoch 5/50
834/834 - 56s - 67ms/step - accuracy: 0.9106 - loss: 0.2363 - val_accuracy: 0.9048 - val_loss: 0.2462
Epoch 6/50
834/834 - 57s - 68ms/step - accuracy: 0.9306 - loss: 0.1873 - val_accuracy: 0.9268 - val_loss: 0.2110
Epoch 7/50
834/834 - 56s - 67ms/step - accuracy: 0.9365 - loss: 0.1667 - val_accuracy: 0.9320 - val_loss: 0.1848
Epoch 8/50
834/834 - 56s - 67ms/step - accuracy: 0.9490 - loss: 0.1343 - val_accuracy: 0.9276 - val_loss: 0.1984
Epoch 9/50
834/834 - 57s - 68ms/step - accuracy: 0.9506 - loss: 0.1308 - val_accuracy: 0.9412 -